# Локализација на робот
### Упатство како да се поврзете со Zethus

- Пробај прво со `p.connect(p.GUI)`. Треба да се рендерира симулацијата во нов прозорец.
- Изврши ја целата целата скрипта
- Отоври Zethus и одбери ги следнте топикс:
    1. Под категоријата Marker имаме топик `/cubes`
    2. Под категоријата Pose имаме топик `/car`
    3. Под категоријата Pose имаме топик `/filter`

In [1]:
import time
import numpy as np
import ipywidgets as ipw
import pybullet_data
import pybullet as p

In [2]:
p.connect(p.GUI) # p.DIRECT ако не работи

0

In [3]:
p.setAdditionalSearchPath(pybullet_data.getDataPath())

## ROS конуникација

In [4]:
import rospy
import jupyros
from geometry_msgs.msg import PoseStamped, Point, Quaternion
from visualization_msgs.msg import Marker
import tf.transformations as tft
rospy.init_node('CarNode')

def get_msg_pose(target):
    """ Returns PoseStamped message with target pose """
    ps = PoseStamped()
    ps.header.stamp = rospy.Time.now()
    ps.header.frame_id = 'Car'
    position, orientation = target
    ps.pose.position = Point(*position)
    ps.pose.orientation = Quaternion(*orientation)
    return ps

def get_msg_marker(marker_id,target):
    """ Returns Marker message with target pose """
    marker = Marker()
    marker.header.stamp = rospy.Time.now();
    marker.ns = "my_namespace";
    marker.id = marker_id;
    marker.type = Marker.CUBE;
    marker.action = Marker.ADD;
    position, orientation = target
    marker.pose.position = Point(*position)
    marker.pose.orientation = Quaternion(*orientation)
    marker.scale.x = 1;
    marker.scale.y = 1;
    marker.scale.z = 1;
    marker.color.a = 1.0; 
    marker.color.r = 0.4117;
    marker.color.g = 0.4117;
    marker.color.b = 0.4117;
    return marker

## Класа за колата во pybullet

In [5]:
class Car:
    def __init__(self, **kwargs):
        self.bid = p.loadURDF('racecar/racecar.urdf', **kwargs)
    
    def set_velocity(self, velocity, force=10):
        """ Set target velocity and force """
        for wheel in [2, 3, 5, 7]:
            p.setJointMotorControl2(
                self.bid, wheel, p.VELOCITY_CONTROL, 
                targetVelocity=velocity, force=force)
    
    def set_steering(self, angle):
        """ Set steering angle """
        for hinge in [4, 6]: 
            p.setJointMotorControl2(
                self.bid, hinge, p.POSITION_CONTROL, targetPosition=angle)
    
    def get_bt_pose(self):
        """ Return car pose """
        return p.getBasePositionAndOrientation(self.bid)
    
    def get_bt_heading(self):
        """ Return heading """
        _,q = p.getBasePositionAndOrientation(self.bid)
        return np.arctan2(2.0*(q[0]*q[1] + q[3]*q[2]), q[3]*q[3] + q[0]*q[0] - q[1]*q[1] - q[2]*q[2])
    
    def get_bt_velocity(self):
        """ Return linear velocity """
        linear,_ = p.getBaseVelocity(car.bid)
        return np.sqrt(linear[0]**2 + linear[1]**2)

## Иницијализација на светот во pybullet

In [6]:
def init_world():
    p.resetSimulation()
    p.setGravity(0, 0, -10)
    p.setRealTimeSimulation(False)
    plane = p.loadURDF('plane.urdf')
    car = Car(basePosition=[1, 1, 0])
    cubes_pos = [(2,2),(4,-3),(-2,-4)] #set cube positions 
    cubes = []
    for i,cube in enumerate(cubes_pos):
        cubes.append(p.loadURDF('cube.urdf', basePosition=[cube[0],cube[1],0.5]))
    return car,cubes

car,cubes = init_world()

## Модел за движење на автомобилот
Автомобилот се управува со вртење на предните гуми додека се движи. Предниот дел на автомобилот се движи во насока во која се насочени тркалата додека се вртат околу задните. Овој едноставен опис е комплициран поради проблеми како што се лизгање поради триење, различното однесување на гумите при различни брзини и потребата надворешната гума да патува во различен радиус од внатрешната гума. Прецизното моделирање на управувањето бара комплициран сет на диференцијални равенки.

![](car.png)

Овде гледаме дека предната гума е насочена во насока $\alpha$ во однос на меѓуоскиното растојание. За краток временски период автомобилот се движи напред, а задното тркало завршува понапред и малку свртено навнатре, како што е прикажано со сината гума. За толку кратка временска рамка, можеме да го приближиме ова како вртење околу радиусот $R$. Можеме да го пресметаме аголот на вртење $\beta$ со

$$\beta = \frac{d}{w} \tan{(\alpha)}$$

а радиусот на вртење R е даден со

$$R = \frac{d}{\beta}$$

каде што растојанието што го поминува задното тркало со оглед на брзината $v$ е $d=v\Delta t$.

Со тоа што $\theta$ е ориентацијата на колата, ја пресметуваме позицијата $C$ пред да започне вртењето како


$$\begin{aligned}
C_x &= x - R\sin(\theta) \\
C_y &= y + R\cos(\theta)
\end{aligned}$$

По движењето напред за време $\Delta t$ новата позиција и ориентација на колата е

$$\begin{aligned} \bar x &= C_x + R\sin(\theta + \beta) \\
\bar y &= C_y - R\cos(\theta + \beta) \\
\bar \theta &= \theta + \beta
\end{aligned}
$$

Откако ќе замениме за $C$ добиваме

$$\begin{aligned} \bar x &= x - R\sin(\theta) + R\sin(\theta + \beta) \\
\bar y &= y + R\cos(\theta) - R\cos(\theta + \beta) \\
\bar \theta &= \theta + \beta
\end{aligned}
$$

### Дизајнирање на состејбените променливи 

За нашиот робот ќе ја задржиме позицијата и ориентацијата:

$$\mathbf x = \begin{bmatrix}x & y & \theta\end{bmatrix}^\mathsf{T}$$

Контролниот влез $\mathbf{u}$ е командната брзина и аголот на управување

$$\mathbf{u} = \begin{bmatrix}v & \alpha\end{bmatrix}^\mathsf{T}$$

In [7]:
def Fx(x, dt, u):
    """ State trainsition model that returns x,y and heading, u is the velocity and steering angle from the car """
    heading = x[2]
    velocity = u[0]
    steering_angle = u[1]
    dist = velocity * dt
    if abs(steering_angle) > 0.001: # is robot turning?
        beta = (dist / wheelbase) * np.tan(steering_angle)
        r = wheelbase / np.tan(steering_angle) # radius
        sinh, sinhb = np.sin(heading), np.sin(heading + beta)
        cosh, coshb = np.cos(heading), np.cos(heading + beta)
        return x + np.array([-r*sinh + r*sinhb, 
                              r*cosh - r*coshb, beta])
    else: # moving in straight line
        return x + np.array([dist*np.cos(heading), dist*np.sin(heading), 0])

### Дизајнирање на мерниот модел

Сензорот обезбедува беринг и растојание до повеќе познати локации во светот. Мерниот модел ке ги конвертира состојбите $\begin{bmatrix}x & y&\theta\end{bmatrix}^\mathsf{T}$ во беринг и ратојание до коцката. Ако $p$ е позиција на обележје, растојанието $r$ е

$$r = \sqrt{(p_x - x)^2 + (p_y - y)^2}$$

Претпоставуваме дека сензорот обезбедува беринг во однос на ориентацијата на роботот, така што мора да ја одземеме ориентацијата на роботот од лежиштето за да го добиеме читањето на сензорот:

$$\phi = \tan^{-1}(\frac{p_y - y}{p_x - x}) - \theta$$

Така нашата мерна функција е

$$\begin{aligned}
\mathbf{z}& = h(\mathbf x, \mathbf P) &+ \mathcal{N}(0, R)\\
&= \begin{bmatrix}
\sqrt{(p_x - x)^2 + (p_y - y)^2} \\
\tan^{-1}(\frac{p_y - y}{p_x - x}) - \theta 
\end{bmatrix} &+ \mathcal{N}(0, R)
\end{aligned}$$

In [8]:
def Hx(x, cubes):
    """ Measurment model that computes distance and bering to an array of cubes """
    hx = []
    for cube in cubes: 
        px, py ,_= p.getBasePositionAndOrientation(cube)[0]
        dist = np.sqrt((px - x[0])**2 + (py - x[1])**2)
        angle = np.arctan2(py - x[1], px - x[0])
        hx.extend([dist, normalize_angle(angle - x[2])])
    return np.array(hx)

## Помошни функции за имплементација на моделот во UKF

In [10]:
def normalize_angle(x):
    """ Returns normalized angle """
    x = x % (2 * np.pi)    # force in range [0, 2 pi)
    if x > np.pi:          # move to [-pi, pi)
        x -= 2 * np.pi
    return x

def residual_h(a, b):
    """ Rewriting residual_h to handle many cubes """
    y = a - b
    # data in format [dist_1, bearing_1, dist_2, bearing_2,...]
    for i in range(0, len(y), 2):
        y[i + 1] = normalize_angle(y[i + 1])
    return y

def residual_x(a, b):
    """ Rewriting residual_x to normalize heading """
    y = a - b
    y[2] = normalize_angle(y[2])
    return y

def state_mean(sigmas, Wm):
    """ Rewriting state_mean to be faster with numpy and compute average of a set of angles """
    x = np.zeros(3)
    sum_sin = np.sum(np.dot(np.sin(sigmas[:, 2]), Wm))
    sum_cos = np.sum(np.dot(np.cos(sigmas[:, 2]), Wm))
    x[0] = np.sum(np.dot(sigmas[:, 0], Wm))
    x[1] = np.sum(np.dot(sigmas[:, 1], Wm))
    x[2] = np.arctan2(sum_sin, sum_cos)
    return x

def z_mean(sigmas, Wm):
    """ Rewriting z_mean to be faster with numpy and compute average of a set of angles """
    z_count = sigmas.shape[1]
    x = np.zeros(z_count)
    for z in range(0, z_count, 2):
        sum_sin = np.sum(np.dot(np.sin(sigmas[:, z+1]), Wm))
        sum_cos = np.sum(np.dot(np.cos(sigmas[:, z+1]), Wm))
        x[z] = np.sum(np.dot(sigmas[:,z], Wm))
        x[z+1] = np.arctan2(sum_sin, sum_cos)
    return x

## Имплементирање на UKF

In [11]:
from filterpy.kalman import MerweScaledSigmaPoints
from filterpy.kalman import UnscentedKalmanFilter as UKF

dt = 1/60 # framerate of the simulation
wheelbase = 0.3 # distance between the cars wheels
sigma_range,sigma_bearing=0.3 , 0.1 # measurement noise

points = MerweScaledSigmaPoints(n=3, alpha=.00001, beta=2, kappa=0, 
                                subtract=residual_x)
ukf = UKF(dim_x=3, dim_z=2*len(cubes), fx=Fx, hx=Hx,
          dt=dt, points=points, x_mean_fn=state_mean, 
          z_mean_fn=z_mean, residual_x=residual_x, 
          residual_z=residual_h)

ukf.x = np.array([1, 1, .1]) # initial car pose
ukf.P = np.diag([.1, .1, .05])
ukf.R = np.diag([sigma_range**2, 
                 sigma_bearing**2]*len(cubes))
ukf.Q = np.eye(3)*0.0001

## Widget за контрола на колата

In [12]:
def cb_steering(change):
    car.set_steering(-change.new)

wd_steering = ipw.FloatSlider(
    value=0, min=-0.5, max=0.5, step=0.05, continuous_update=True, 
    description='Волан:')
wd_steering.observe(cb_steering, names='value')


def cb_velocity(change):
    car.set_velocity(change.new)

wd_velocity = ipw.IntSlider(
    value=0, min=-10, max=30, step=1, continuous_update=True, 
    description='Брзина:')
wd_velocity.observe(cb_velocity, names='value')

ipw.VBox([wd_steering, wd_velocity])

## Нишка за симулација во pybullet и обновување на филтерот

In [13]:
%%thread_cell

rate = rospy.Rate(60)
# create rospy publishers so send data to zethus
pub_car = rospy.Publisher('/car', PoseStamped, queue_size=10)
pub_filter = rospy.Publisher('/filter', PoseStamped, queue_size=10)
vis_pub = rospy.Publisher('/cubes', Marker, queue_size=10)
while True:
    p.stepSimulation()
    ukf.predict(u=(car.get_bt_velocity(),wd_steering.value))
    x,y,_ = car.get_bt_pose()[0] # get current pose of the car 
    z = []
    for cube in cubes: # simulate reading from sensor for each cube
        px, py ,_= p.getBasePositionAndOrientation(cube)[0]
        distance = np.sqrt((px - x)**2 + (py - y)**2) + np.random.randn()*sigma_range # add noise to the distance
        bearing = np.arctan2(py - y, px - x)
        angle = (normalize_angle(bearing - car.get_bt_heading() + np.random.randn()*sigma_bearing)) # add noise to the bering
        z.extend([distance, angle])
        vis_pub.publish(get_msg_marker(cube,p.getBasePositionAndOrientation(cube))) # send curent pose of each cube
        
    ukf.update(z,cubes=cubes)
    # send real and filtered pose
    pub_car.publish(get_msg_pose(car.get_bt_pose()))
    euler = p.getEulerFromQuaternion(car.get_bt_pose()[1])
    pub_filter.publish(get_msg_pose([(ukf.x[0], ukf.x[1], 0), p.getQuaternionFromEuler([euler[0], euler[1], ukf.x[2]])] ))
    rate.sleep()

Output(layout=Layout(border='1px solid gray'))